In [ ]:
%matplotlib inline 
from matplotlib import pyplot as plt

import cv2
import numpy as np
import math
import os

def detect_edges(frame):
    """
    Detects edges in the input frame using Canny edge detection.

    Args:
        frame (ndarray): Frame captured from the USB camera.

    Returns:
        edges (ndarray): Edge-detected frame.
    """
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian Blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Perform Canny edge detection
    edges = cv2.Canny(blurred, threshold1=50, threshold2=150)

    return edges

def calculate_angle(edges):
    """
    Calculates the direction for a car to travel to stay in the center of the detected edges.

    Args:
        edges (ndarray): Edge-detected frame.

    Returns:
        angle (float): Angle (in degrees) for the car to travel.
    """
    # Get the dimensions of the frame
    height, width = edges.shape

    # Find the coordinates of the edge pixels
    edge_points = np.column_stack(np.where(edges > 0))

    if len(edge_points) == 0:
        return None  # No edges detected

    # Calculate the centerline of the detected edges
    left_edge_x = edge_points[edge_points[:, 1] < width // 2][:, 1]
    right_edge_x = edge_points[edge_points[:, 1] >= width // 2][:, 1]

    if len(left_edge_x) == 0 or len(right_edge_x) == 0:
        return None  # Edges do not span both sides

    left_x = np.mean(left_edge_x)
    right_x = np.mean(right_edge_x)

    # Calculate the target center x-coordinate
    center_x = (left_x + right_x) / 2

    # Compute the angle to adjust direction (relative to the center of the frame)
    offset = center_x - (width / 2)
    angle = math.degrees(math.atan2(offset, height))

    return angle